## Урок 2. Создание признакового пространства

1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
 - Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
 - Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
 - Исключим стоп-слова с помощью stop_words='english'. 
 - Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().
 
2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
 - Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
 - Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
 - Исключим стоп-слова с помощью stop_words='english'.
 - Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().
 
3. Проверьте ваши векторайзеры на корпусе который использовали на вебинаре, составьте таблицу метод векторизации и скор который вы получили (в методах векторизации по изменяйте параметры что бы добиться лучшего скора) обратите внимание как падает/растёт скор при уменьшении количества фичей, и изменении параметров, так же попробуйте применить к векторайзерам PCA для сокращения размерности посмотрите на качество сделайте выводы.

### Дополнительные задания из старого файла.  

4. Натренируем gensim.models.Word2Vec модель на наших данных.
Тренировать будем на токенизированных твитах combine_df['tweet_token']
Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.
Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.


5. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".


6. Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).
Давайте проверим векторное представление любого слова из нашего корпуса, например "food".


7. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.
Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size)) и поделить финальный вектор на количество слов в твите.

На выходе должен получиться wordvec_df.shape = (49159, 200).

## Решение.

### Обработанные данные открыть из файла который мы сохранили в прошом ДЗ.

sklearn.feature_extraction.text.CountVectorizer.fit_transform()

1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
 - Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
 - Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
 - Исключим стоп-слова с помощью stop_words='english'.
 - Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().

In [1]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
with open('combine_df.pickle', 'rb') as handle:
    df = pickle.load(handle)
df.head(3)

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"


Соберем все уникальные слова из 4 загруженных предложений, игнорируя регистр, пунктуацию и односимвольные токены. Это и будет наш словарь (известные слова).

In [4]:
df['tweet_stemmed'] = df['tweet_stemmed'].apply(lambda x:' '.join(x))

In [5]:
df['tweet_lemmatized'] = df['tweet_lemmatized'].apply(lambda x:' '.join(x))

In [6]:
count_vectorizer = CountVectorizer(ngram_range=(1, 1),
                                   analyzer='word',
                                   binary=False,
                                   max_df=0.9,
                                   stop_words='english',
                                   max_features=1000)

In [7]:
# Создаем the Bag-of-Words модель
tweet_stemmed_bow = count_vectorizer.fit_transform(df['tweet_stemmed'].values)

In [8]:
# Отобразим Bag-of-Words модель как DataFrame
feature_names = count_vectorizer.get_feature_names()
tweet_stemmed_df = pd.DataFrame(tweet_stemmed_bow.toarray(), columns = feature_names)

tweet_stemmed_df.head(3)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Создаем the Bag-of-Words модель
tweet_lemmatized_bow = count_vectorizer.fit_transform(df['tweet_lemmatized'].values)

In [10]:
# Отобразим Bag-of-Words модель как DataFrame
feature_names = count_vectorizer.get_feature_names()
tweet_lemmatized_df = pd.DataFrame(tweet_lemmatized_bow.toarray(), columns = feature_names)

tweet_lemmatized_df.head(3)

,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,...,year,yes,yesterday,yo,yoga,york,young,youtube,yr,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
 - Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
 - Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
 - Исключим стоп-слова с помощью stop_words='english'.
 - Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().

In [11]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1),
                                   analyzer='word',
                                   binary=False,
                                   max_df=0.9,
                                   stop_words='english',
                                   max_features=1000)

In [12]:
# Создаем the Bag-of-Words модель
tweet_stemmed_bow_tfidf = count_vectorizer.fit_transform(df['tweet_stemmed'].values)

In [13]:
# Отобразим Bag-of-Words модель как DataFrame
feature_names = count_vectorizer.get_feature_names()
tweet_stemmed_tfidf_df = pd.DataFrame(tweet_stemmed_bow_tfidf.toarray(), columns = feature_names)

tweet_stemmed_tfidf_df.head(3)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Создаем the Bag-of-Words модель
tweet_lemmatized_bow_tfidf = count_vectorizer.fit_transform(df['tweet_lemmatized'].values)

In [15]:
# Отобразим Bag-of-Words модель как DataFrame
feature_names = count_vectorizer.get_feature_names()
tweet_lemmatized_tfidf_df = pd.DataFrame(tweet_lemmatized_bow_tfidf.toarray(), columns = feature_names)

tweet_lemmatized_tfidf_df.head(3)

,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,...,year,yes,yesterday,yo,yoga,york,young,youtube,yr,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


3. Проверьте ваши векторайзеры на корпусе который использовали на вебинаре.<br> 
Составьте таблицу метод векторизации и скор который вы получили (в методах векторизации по изменяйте параметры что бы добиться лучшего скора).<br>
 Обратите внимание как падает/растёт скор при уменьшении количества фичей, и изменении параметров, <br>
 Так же попробуйте применить к векторайзерам **PCA** для сокращения размерности посмотрите на качество сделайте выводы.

In [16]:
tweet_stemmed_df
tweet_lemmatized_df

tweet_stemmed_bow_tfidf
tweet_lemmatized_tfidf_df

,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,...,year,yes,yesterday,yo,yoga,york,young,youtube,yr,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# Загружаем данные
data = open('corpus').read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append(content[0])
    texts.append(" ".join(content[1:]))

# создаем df
trainDF = pd.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels
trainDF.head(5)

,text,label
0,Stuning even for the non-gamer: This sound tra...,__label__2
1,The best soundtrack ever to anything.: I'm rea...,__label__2
2,Amazing!: This soundtrack is my favorite music...,__label__2
3,Excellent Soundtrack: I truly like this soundt...,__label__2
4,"Remember, Pull Your Jaw Off The Floor After He...",__label__2


In [18]:
from sklearn import model_selection, preprocessing, linear_model
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

In [19]:
# Создадим словарь, в котрый будем записывать результаты рабты. 

vectorizes_table_compare = {'vectorizer':[],
'features':[], 
'score': [],
'pca_components':[],
'pca_score': []
}

In [20]:
vectorizes_table_compare.keys()

dict_keys(['vectorizer', 'features', 'score', 'pca_components', 'pca_score'])

In [21]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

n_teatures_list = [100, 200,300,500,800,1000,None]

## CountVectorizer

In [22]:
for n_features in n_teatures_list:
    count_vect = CountVectorizer(analyzer='word',
                                token_pattern=r'\w{1,}',
                                max_df=0.9,
                                stop_words='english',
                                max_features=n_features)

    count_vect.fit(trainDF['text'])

    xtrain_count =  count_vect.transform(train_x)
    xvalid_count =  count_vect.transform(valid_x)

    classifier = linear_model.LogisticRegression()
    classifier.fit(xtrain_count, train_y)
    predictions = classifier.predict(xvalid_count)

    score = accuracy_score(valid_y, predictions)
    # print(score)
    vectorizes_table_compare['score'].append(score)
    vectorizes_table_compare['vectorizer'].append('CountVectorizer')
    vectorizes_table_compare['features'].append(n_features)
 #=============================================================
    if n_features == 100:
       n_components=100
    else:
       n_components=200 

    pca = PCA(n_components=n_components)
    X_train = pca.fit_transform(xtrain_count.toarray())
    X_test = pca.transform(xvalid_count.toarray())

    classifier = linear_model.LogisticRegression()
    classifier.fit(X_train, train_y)
    predictions = classifier.predict(X_test)

    score = accuracy_score(valid_y, predictions)
    vectorizes_table_compare['pca_components'].append(n_components)
    vectorizes_table_compare['pca_score'].append(score)

## HashingVectorizer

In [ ]:
for n_features in n_teatures_list:
    
    if n_features is None:
        n_features= 10000   # 1048576
    vct = HashingVectorizer(analyzer='word', 
                            token_pattern=r'\w{1,}', 
                            n_features=n_features) 
 
    vct.fit(trainDF['text'])
    xtrain_hash =  vct.transform(train_x)
    xvalid_hash =  vct.transform(valid_x)

    classifier = linear_model.LogisticRegression()
    classifier.fit(xtrain_hash, train_y)
    predictions = classifier.predict(xvalid_hash)

    score = accuracy_score(valid_y, predictions)

    vectorizes_table_compare['score'].append(score)
    vectorizes_table_compare['vectorizer'].append('HashingVectorizer')
    vectorizes_table_compare['features'].append(n_features)
 #=============================================================
    if n_features == 100:
       n_components=100
   #  elif n_components is None:
   #     continue
    else:
       n_components=200 

    pca = PCA(n_components=n_components)
    X_train = pca.fit_transform(xtrain_hash.toarray())
    X_test = pca.transform(xvalid_hash.toarray())

    classifier = linear_model.LogisticRegression()
    classifier.fit(X_train, train_y)
    predictions = classifier.predict(X_test)

    score = accuracy_score(valid_y, predictions)
    vectorizes_table_compare['pca_components'].append(n_components)
    vectorizes_table_compare['pca_score'].append(score)

## TfidfVectorizer

In [25]:
for n_features in n_teatures_list:
    tfidf_vectorizer = TfidfVectorizer(
                                    analyzer='word',
                                    token_pattern=r'\w{1,}',
                                    binary=False,
                                    max_df=0.9,
                                    stop_words='english',
                                    max_features=n_features,
                                    )

    tfidf_vectorizer.fit(trainDF['text'])

    xtrain_tfidf =  tfidf_vectorizer.transform(train_x)
    xvalid_tfidf =  tfidf_vectorizer.transform(valid_x)

    classifier = linear_model.LogisticRegression()
    classifier.fit(xtrain_tfidf, train_y)
    predictions = classifier.predict(xvalid_tfidf)
    # predictions
    score = accuracy_score(valid_y, predictions)
    
    vectorizes_table_compare['vectorizer'].append('TfidfVectorizer')
    vectorizes_table_compare['features'].append(n_features)
    vectorizes_table_compare['score'].append(score)
    
#=============================================================
    if n_features == 100:
       n_components=100
    else:
       n_components=200 

    pca = PCA(n_components=n_components)
    X_train = pca.fit_transform(xtrain_tfidf.toarray())
    X_test = pca.transform(xvalid_tfidf.toarray())

    classifier = linear_model.LogisticRegression()
    classifier.fit(X_train, train_y)
    predictions = classifier.predict(X_test)

    score = accuracy_score(valid_y, predictions)
    vectorizes_table_compare['pca_components'].append(n_components)
    vectorizes_table_compare['pca_score'].append(score)

In [26]:
result_df = pd.DataFrame(vectorizes_table_compare)
result_df

,vectorizer,features,score,pca_components,pca_score
0,CountVectorizer,100.0,0.7480,100,0.7480
1,CountVectorizer,200.0,0.7948,200,0.7948
2,CountVectorizer,300.0,0.8000,200,0.7964
3,CountVectorizer,500.0,0.8144,200,0.8036
4,CountVectorizer,800.0,0.8184,200,0.8128
5,CountVectorizer,1000.0,0.8156,200,0.8084
6,CountVectorizer,NaN,0.8484,200,0.8184
7,HashingVectorizer,100.0,0.6652,100,0.6652
8,HashingVectorizer,200.0,0.7112,200,0.7116
9,HashingVectorizer,300.0,0.7252,200,0.7204


In [27]:
list(result_df.groupby('features'))

[(100.0,
             vectorizer  features   score  pca_components  pca_score
  0     CountVectorizer     100.0  0.7480             100     0.7480
  7   HashingVectorizer     100.0  0.6652             100     0.6652
  14    TfidfVectorizer     100.0  0.7504             100     0.7504),
 (200.0,
             vectorizer  features   score  pca_components  pca_score
  1     CountVectorizer     200.0  0.7948             200     0.7948
  8   HashingVectorizer     200.0  0.7112             200     0.7116
  15    TfidfVectorizer     200.0  0.7984             200     0.7984),
 (300.0,
             vectorizer  features   score  pca_components  pca_score
  2     CountVectorizer     300.0  0.8000             200     0.7964
  9   HashingVectorizer     300.0  0.7252             200     0.7204
  16    TfidfVectorizer     300.0  0.8020             200     0.8008),
 (500.0,
             vectorizer  features   score  pca_components  pca_score
  3     CountVectorizer     500.0  0.8144             200    

**Выводы:** Видно что с ростом к-ва фитчей качество обучения растет. 
В плане качества предсказания мета распределились в порядке убывания:
1. TfidfVectorizer,
2. CountVectorizer,
3. HashingVectorizer

===============================================================================

### Дополнительные задания.(из другого файла). 

4. Натренируем gensim.models.Word2Vec модель на наших данных.
Тренировать будем на токенизированных твитах combine_df['tweet_token']
Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.
Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.


5. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".


6. Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).
Давайте проверим векторное представление любого слова из нашего корпуса, например "food".


7. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.
Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size)) и поделить финальный вектор на количество слов в твите.

На выходе должен получиться wordvec_df.shape = (49159, 200).

## Решение:

4. Натренируем **gensim.models.Word2Vec** модель на наших данных.

Тренировать будем на токенизированных твитах combine_df['tweet_token']

Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.

Используем функцию **train()** с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.

https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial

In [327]:
# !pip install gensim

In [328]:
# !pip install python-Levenshtein

In [329]:
from gensim.models import Word2Vec

In [330]:
# sentences[0]
data_train = df['tweet_token']

In [332]:
from gensim.test.utils import datapath
from gensim import utils

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = datapath('lee_background.cor')
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)


The parameters:

    min_count = int - Ignores all words with total absolute frequency lower than this - (2, 100)

    window = int - The maximum distance between the current and predicted word within a sentence. E.g. window words on the left and window words on the left of our target - (2, 10)

    size = int - Dimensionality of the feature vectors. - (50, 300)

    sample = float - The threshold for configuring which higher-frequency words are randomly downsampled. Highly influencial. - (0, 1e-5)

    alpha = float - The initial learning rate - (0.01, 0.05)

    min_alpha = float - Learning rate will linearly drop to min_alpha as training progresses. To set it: alpha - (min_alpha * epochs) ~ 0.00

    negative = int - If > 0, negative sampling will be used, the int for negative specifies how many "noise words" should be drown. If set to 0, no negative sampling is used. - (5, 20)

    workers = int - Use these many worker threads to train the model (=faster training with multicore machines)



In [335]:
w2v_model = Word2Vec(#sentences=data_train, 
                    vector_size=200, 
                    window=5, 
                    min_count=2, 
                    sg = 1, 
                    hs = 0, 
                    negative = 10, 
                    workers= 32, 
                    seed = 34
                   )

In [336]:
from time import time  # To time our operations

In [337]:
t = time()

w2v_model.build_vocab(data_train, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.01 mins


In [338]:
t = time()

w2v_model.train(data_train, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 0.88 mins


In [339]:
# найдем top-3 самых близких слов
result = w2v_model.wv.similar_by_word("bihday", topn=3)
print(result)

result = w2v_model.wv.similar_by_word("man", topn=3)
print(result)

result = w2v_model.wv.similar_by_word("cat", topn=3)
print(result)

result = w2v_model.wv.similar_by_word("mouth", topn=3)
print(result)

[('bihay', 0.5713201761245728), ('bihdaycake', 0.5701730847358704), ('anniversaire', 0.5558146238327026)]
[('cautious', 0.4471203088760376), ('gokaldas', 0.4262577295303345), ('hindu', 0.4240967333316803)]
[('kitty', 0.5753360390663147), ('catsofinstagram', 0.5603662133216858), ('grumpycat', 0.5599806308746338)]
[('endlessly', 0.5930461287498474), ('conormcgregor', 0.5641347765922546), ('godislove', 0.5115699768066406)]


5. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".

In [340]:
w2v_model.wv.most_similar(positive=["dinner"])

[('bihdaydinner', 0.514960527420044),
 ('shawarma', 0.5066928267478943),
 ('cookout', 0.5062406063079834),
 ('bolognese', 0.5029006004333496),
 ('spaghetti', 0.49887171387672424),
 ('foodblogger', 0.4981864094734192),
 ('sizzle', 0.4914211630821228),
 ('tacotuesday', 0.48558303713798523),
 ('hamburger', 0.4844452142715454),
 ('sissy', 0.4799242317676544)]

In [341]:
w2v_model.wv.most_similar(positive=["trump"])

[('donald', 0.5562459826469421),
 ('impeachment', 0.5360007286071777),
 ('suppoer', 0.5199249982833862),
 ('donaldtrump', 0.5138074159622192),
 ('dumptrump', 0.5079053044319153),
 ('racists', 0.5041542649269104),
 ('cuck', 0.5005506277084351),
 ('unfit', 0.49091798067092896),
 ('crony', 0.4903233051300049),
 ('fuhered', 0.4902151823043823)]

6. Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).
Давайте проверим векторное представление любого слова из нашего корпуса, например "food".

In [345]:
w2v_model.wv['food']

array([ 9.72253382e-01, -5.32931387e-01, -1.97032005e-01, -3.45115751e-01,
       -8.54220331e-01, -3.66696268e-01, -5.76752424e-01, -3.21528465e-01,
       -3.21828604e-01, -1.38551265e-01, -5.17741218e-02,  1.44866526e-01,
       -5.31215966e-01,  2.91604221e-01,  2.50220120e-01, -3.99498105e-01,
        6.26124084e-01, -5.11294961e-01, -1.03406668e+00, -3.53472352e-01,
       -4.64780629e-01,  4.70135100e-02, -1.01344299e+00, -7.69010335e-02,
        8.42454806e-02,  4.17801142e-01, -5.29752970e-01, -4.05228995e-02,
        7.05603004e-01, -4.64020576e-03,  6.14214182e-01, -4.81294423e-01,
       -1.36772871e-01,  1.36004180e-01, -1.19141340e+00,  3.53754796e-02,
        5.39124966e-01, -2.91756272e-01,  4.25235897e-01, -1.78252921e-01,
       -4.23755720e-02, -1.91590205e-01, -2.74610192e-01,  3.51258308e-01,
       -5.27885556e-01, -1.68294668e-01, -6.34512603e-01, -3.26083809e-01,
       -5.06048024e-01, -3.28379840e-01,  9.41767275e-01,  5.38997293e-01,
       -1.44808456e-01, -

7. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. 
Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите.
Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. 
Теперь у нас есть 200 функций word2vec для наших данных.
Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите.
В цикле сделать:  vec += model_w2v[word].reshape((1, size)) и поделить финальный вектор на количество слов в твите.

На выходе должен получиться wordvec_df.shape = (49159, 200).

In [342]:
def tweet_mean_vec_value(tweet_words):
  
    meen_values = []
    word_count = len(tweet_words)
 # получим теперь векторы для нашего твита. 
 # print(tweet_words)
    tweet_vectors = []
    for word in tweet_words:
        try:
            tweet_vectors.append(w2v_model.wv[word])
        except Exception as ex:
            word_count-=1
            # print(str(ex))
            
    for i in range(200):
        meen_value=0
        for j in range(word_count):
            meen_value += tweet_vectors[j][i]
        if word_count>0:
            meen_values.append(meen_value/word_count)
    # print('======================')
    if len(meen_values)>0:
        return meen_values

In [343]:
tweets_vectors = list(map(tweet_mean_vec_value,df['tweet_token'].values))
# перепишем функцию. для всех 

In [ ]:
np.array(tweets_vectors).shape

(49159,)

In [346]:
tweets = []
nones = 0
for t in tweets_vectors:
    if t is None:
        nones+=1
    elif len(t) < 200:
        print(len(t))
    else:
        tweets.append(t)
        
print('None_count=',nones)
print(np.array(tweets).shape)

None_count= 20
(49139, 200)


Странно, но в некоторых витах есть слова для которых небыло создано вектора. 
Поэтому при ображении к такому слову возникает ошибка. 
При этом есть 20 твитов, ни для одного слова в которомнебыло создано вектора, таких твитов 20. это пустые массивы. 
Почему так вышло? 